In [11]:
!pip install weaviate-client
!pip install openai
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Set the OpenAI key and current Weaviate IP to run this notebook
OPENAI_KEY = os.getenv("OPENAI_KEY")
WEAVIATE_IP_ADDRESS = "34.42.138.162"

In [18]:
import weaviate
import csv
from os import listdir
from datetime import datetime, timezone

# Initialize Weaviate client
client = weaviate.Client(
    # Weaviate instance URL
    url="http://" + WEAVIATE_IP_ADDRESS + ":8080",
    additional_headers={
        # Replace with your OpenAI key
        "X-OPENAI-Api-Key": OPENAI_KEY,
    }
)

In [19]:
client.schema.get()

{'classes': [{'class': 'Document',
   'description': 'A full document of text from a scraped webpage with full details.',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'indexTimestamps': True,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {'model': 'gpt-3.5-turbo'},
    'text2vec-openai': {'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the document.',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'text',
     'tokenization': 'whitespace'},
    {'dataType': ['text'],
     'description': 'The address of the website this document comes from.',
     'indexFilterable': True,
    

In [30]:
def get_all_website_addresses(client):
    # Construct the GraphQL query to fetch all websiteAddress values from the Pages class
    graphql_query = '''
    {
        Get {
            Pages {
                websiteAddress
            }
        }
    }
    '''

    # Initialize a set to collect unique website addresses
    website_addresses_set = set()

    try:
        # Perform the query
        result = client.query.raw(graphql_query)

        # Extract website addresses from the 'Pages' class results
        pages = result.get('data', {}).get('Get', {}).get('Pages', [])
        for page in pages:
            if 'websiteAddress' in page and page['websiteAddress']:
                # Add to set to ensure uniqueness
                website_addresses_set.add(page['websiteAddress'])

        # Convert the set back to a list to return
        return list(website_addresses_set)

    except Exception as e:
        print(f"An error occurred while getting website addresses: {e}")
        return []

# Example usage: Replace `client` with your actual Weaviate client instance.
website_addresses = get_all_website_addresses(client)
print(website_addresses)


['ai21.com']


In [31]:
import weaviate

def get_all_timestamps_for_website(client, website_address):
    # GraphQL query that fetches timestamps for a particular websiteAddress
    graphql_query = f'''
    {{
        Get {{
            Pages(
                where: {{
                    operator: Equal
                    path: ["websiteAddress"]
                    valueString: "{website_address}"
                }}
            ) {{
                timestamp
            }}
        }}
    }}
    '''
    
    # Initialize a set to collect unique timestamps
    timestamps_set = set()

    try:
        # Perform the query
        result = client.query.raw(graphql_query)

        # Extract timestamps from the 'Pages' class results
        pages = result.get('data', {}).get('Get', {}).get('Pages', [])
        for page in pages:
            if 'timestamp' in page and page['timestamp']:
                # Add to set to ensure uniqueness
                timestamps_set.add(page['timestamp'])

        # Convert the set back to a list to return
        return list(timestamps_set)

    except Exception as e:
        print(f"An error occurred while getting timestamps: {e}")
        return []

# Example usage:
# REPLACE_WITH_WEAVIATE_IP = "127.0.0.1"
# WEBSITE_ADDRESS_TO_QUERY = "http://example.com"
timestamps = get_all_timestamps_for_website(client, "ai21.com")
print(timestamps)


['2023-10-06T18-11-24']
